In [ ]:
# TCN 15 YEAR DATA NO SENTIMENT
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tcn import TCN  # Make sure to install via: pip install keras-tcn

# ----------------------------
# 1. Data Loading and Preprocessing
# ----------------------------
# Load BTC data and sort by datetime
df_btc = pd.read_csv('/notebooks/BITCOIN_DATA.csv', parse_dates=['datetime'])
df_btc.sort_values('datetime', inplace=True)
# df_btc = df_btc[df_btc['datetime'] <= '2019-05-01'].copy()
df_btc = df_btc[['datetime', 'close', 'volume']]
print(df_btc.head())

# Split data into training and testing sets
train_df = df_btc[(df_btc['datetime'] >= '2011-01-01') & (df_btc['datetime'] < '2023-12-31')].copy()
test_df = df_btc[df_btc['datetime'] >= '2024-01-01'].copy()

# Use separate scalers for close and volume to simplify inverse transformation
close_scaler = MinMaxScaler()
volume_scaler = MinMaxScaler()

train_df['close_scaled'] = close_scaler.fit_transform(train_df[['close']])
train_df['volume_scaled'] = volume_scaler.fit_transform(train_df[['volume']])
test_df['close_scaled'] = close_scaler.transform(test_df[['close']])
test_df['volume_scaled'] = volume_scaler.transform(test_df[['volume']])

# Create numpy arrays combining the two features (order: [close, volume])
train_scaled = train_df[['close_scaled', 'volume_scaled']].values
test_scaled = test_df[['close_scaled', 'volume_scaled']].values

# ----------------------------
# 2. Creating Sequences for Time Series
# ----------------------------
lookback = 30  # Use previous 30 days to predict the next day's close

def create_sequences(data, lookback):
    X, y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i - lookback:i])
        y.append(data[i, 0])  # Target is the 'close' price (first column)
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test = create_sequences(test_scaled, lookback)

# ----------------------------
# 3. Building and Training the TCN Model
# ----------------------------
# Build the TCN model; here we use two TCN layers stacked with a Dense output
model = Sequential()
model.add(TCN(nb_filters=64, kernel_size=3, dropout_rate=0.2, return_sequences=True, input_shape=(lookback, 2)))
model.add(TCN(nb_filters=64, kernel_size=3, dropout_rate=0.2, return_sequences=False))
model.add(Dense(1))  # Predict the close price
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1)

# ----------------------------
# 4. Prediction and Inverse Transformation
# ----------------------------
predictions_scaled = model.predict(X_test)
prediction_tcn = close_scaler.inverse_transform(predictions_scaled)
y_test_actual_tcn = close_scaler.inverse_transform(y_test.reshape(-1, 1))

# Align the dates (accounting for the lookback period)
test_dates = test_df['datetime'].values[lookback:]

# ----------------------------
# 5. Visualization: Predicted vs Actual Close Prices
# ----------------------------
plt.figure(figsize=(12, 6))
plt.plot(test_dates, y_test_actual_tcn, label='Actual Close Price', color='blue')
plt.plot(test_dates, prediction_tcn, label='Predicted Close Price (TCN)', color='red', linestyle='--')
plt.xlabel('Date')
plt.ylabel('BTC Close Price')
plt.title('BTC Close Price Prediction: Actual vs Predicted (Mar 15 - May 1, 2019)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# ----------------------------
# 6. Performance Metrics
# ----------------------------
mse = mean_squared_error(y_test_actual_tcn, prediction_tcn)
mae = mean_absolute_error(y_test_actual_tcn, prediction_tcn)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test_actual_tcn, prediction_tcn) * 100
r2 = r2_score(y_test_actual_tcn, prediction_tcn)

print("Performance Metrics for the TCN Model:")
print(f"MSE: {mse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")
print(f"R2 Score: {r2:.2f}")
